In [72]:
#coding=gbk
import math
import random
import matplotlib.pyplot as plt
import time
import numpy as np
from decimal import *
getcontext().prec=36

class T_kernel_SGD():
    def __init__(self,d,s):
        self.d = d
        self.s = s
        self.Tn = [np.array([1.])]
        self.K_n = [[np.array([0.])]]
        self.K_t = [[ np.array([1.]) ]]
        self.L_n = 0
        self.fn = [[np.array([0.])]]
        self.f_average_n = [[np.array([0.])]]
        self.computing_ploy = []
        self.gamma0 = 1 #Initial step size
        self.t = 0 #Step decay rate
        self.epoch = 1 #  Current number of iterations
        self.theta = 1/(1+2*self.s)
        self.dim_Pi_Ln_d = 1
        self.cc_B4 = -math.pi**4/48
        
    def factorial(self,a,b,i,model):#i是间隔的步长,model是输出模式，输出列表或值
        if model == 'list':
            S = 1
            Output = []
            for k in range(0,b,i):
                S *= (k+a)
                Output.append(S)
        elif model ==  'value':
            S = 1
            for k in range(0,b,i):
                S *= (k+a)
            Output = S
        return Output
    
    def jiecheng(self,x):
        S = 1
        for i in range(1,x+1):
            S *= i
        return S
    
    def jiecheng_list(self,x):
        S = 1
        Output = [1]
        for i in range(1,x+1):
            S *= i
            Output.append(S)
        return Output
            
            
    
    def combination_num(self,a,b):#C^a_b
        if a == -1:
            C = 0
        elif a == 0:
            C = 1
        else:
            C = self.jiecheng(b)//(self.jiecheng(a)*self.jiecheng(b-a))
        return C
    
    def Multivariate_binomial(self,d,k,i):
        #self.T_n  is a polynomial before
        #k is the order of polynomial which will be calculating
        #d is dimension i in the element
        self.T_L_n = []
        if k == 1:
            self.T_L_n = np.array([1.])
        else:
            for a in self.Tn[i:]:
                self.T_L_n = np.append(self.T_L_n,a)
            Dim_list = [0]+[self.combination_num(l,d+l-1) for l in range(k)]
            for l in range(1,len(Dim_list)):
                for p in range(Dim_list[l-1],Dim_list[l]):
                    self.T_L_n[p] = self.T_L_n[p]*(k/(k-l+1))
        
        
    def updating_Zt(self):
        self.K_n = [[np.array([0.])]]
        if self.L_n >= 1:
            self.Tn = [np.array([1.])]
            omega_Ln = (self.dim_Pi_Ln_d)**(-2*self.s)
            fact1 = self.factorial(self.d/2,self.L_n,1,'value')#(d/2)k
            fact2 = [1]+self.factorial(2-self.L_n-self.d/2,int(self.L_n/2),1,'list')#(2-k-d/2)j
            fact3 = self.jiecheng_list(int(self.L_n/2))#(j)!
            fact4 = self.jiecheng_list(self.L_n)#(k-2j)!
            fact5 = [2**l for l in range(self.L_n+1)]
            #print("update Ln","1,",fact1,fact2,"3",fact3,fact4,self.L_n)
            Coe = [0 for l in range(self.L_n+1)]
            if self.L_n%2 == 0:
                for i in range(self.L_n+1):
                    if i%2 == 0:
                        Coe[i] = omega_Ln*fact1*fact5[i]/(fact2[-i//2-1]*fact3[-i//2-1]*fact4[i])
                    else:
                        Coe[i] = 0
            elif self.L_n%2 == 1:
                for i in range(self.L_n+1):
                    if i%2 == 1:
                        Coe[i] = omega_Ln*fact1*fact5[i]/(fact2[-(i+1)//2]*fact3[-(i+1)//2]*fact4[i])
                    else:
                        Coe[i] = 0
            #print(omega_Ln)
            #print(self.L_n,"Coe",Coe)            
            self.K_n[0][0][0] += Coe[0]*1   
            for k in range(1,self.L_n+1):
                self.Tn_lin = []
                for i in range(self.d):
                    self.Multivariate_binomial(self.d-i,k,i)
                    self.Tn_lin.append(self.T_L_n[:])
                self.Tn = self.Tn_lin[:]
                #print("L_n",self.L_n,k,self.Tn)
                self.K_n.append([ Coe[k]*A for A in self.Tn])
            self.K_t.append([np.zeros(len(A)) for A in self.Tn])
            self.K_t = [[a+b for a,b in zip(A,B)] for A,B in zip(self.K_t,self.K_n)]
            
    def Computation_ploynomial(self,x,order):#Note that here order is the order of the polynomial
        self.Tn = [np.array([1.])]
        if order == 0:
            self.computing_ploy = [self.Tn[:]]
        if order >= 1:
            self.computing_ploy = [self.Tn[:]]
            self.Tn = [ np.array([x[i]]) for i in range(self.d)]
            self.computing_ploy.append(self.Tn[:])
            for i in range(2,order+1):
                self.Tn_lin = []
                B = []
                for j in range(self.d-1,-1,-1):
                    B = np.append(self.Tn[j],B)
                    self.Tn_lin.insert(0,x[j]*B)
                self.Tn = self.Tn_lin[:]
                self.computing_ploy.append(self.Tn[:])
            
    def Computation_K_Xnt(self,X):
        self.Computation_ploynomial(X,self.L_n)
        self.K_Xnt_list = [[ a*b for a,b in zip(A,B)] for A,B in zip(self.computing_ploy,self.K_t)]     
    
    def Computation_fn(self,X):
        self.Computation_ploynomial(X,self.L_n)
        return sum([sum([ np.dot(a,b) for a,b in zip(A,B)])for A,B in zip(self.computing_ploy,self.fn)])
    
    def Computation_fn_average(self,X):
        self.Computation_ploynomial(X,self.L_n)
        return sum([sum([ np.dot(a,b) for a,b in zip(A,B)])for A,B in zip(self.computing_ploy,self.f_average_n)])
    
    def iteration(self,X,Y):
        coe = self.gamma0*((self.epoch)**(-self.t))*(Y - self.Computation_fn(X))
        if self.dim_Pi_Ln_d+1 < self.epoch**self.theta:
            self.L_n += 1 
            self.dim_Pi_Ln_d = self.combination_num(self.L_n,self.L_n+self.d-1)+self.combination_num(self.L_n-1,self.L_n+self.d-2)
            self.updating_Zt()
            A = [np.zeros(len(B)) for B in self.Tn]
            self.fn.append(A[:])
            self.f_average_n.append(A[:])
        self.Computation_K_Xnt(X)
        self.fn= [[ a+coe*b for a,b in zip(A,B)] for A,B in zip(self.fn,self.K_Xnt_list)] 
        self.f_average_n = [[ (self.epoch*a+b)/(self.epoch+1) for a,b in zip(A,B)] for A,B in zip(self.f_average_n,self.fn)]
    """    
    def construct_data(self):
        X = [random.gauss(0,1) for i in range(self.d)]
        mo = np.sqrt(sum([i**2 for i in X]))
        X = [i/mo for i in X]
        Y = np.sin(sum(X))
        Z = Y+random.gauss(0,0.05)
        return [X,Z,Y]
        """
    
    def construct_data(self):
        X = [random.uniform(0,1) for i in range(self.d-1)]
        Y = 10*np.sin(X[0]*X[1])+20*(X[2]-0.5)**2+10*X[3]+5*X[4]+X[1]*X[2]*X[3]+np.cos(X[2]*X[3])
        Z = Y+random.gauss(0,0.5)
        X = [ 0.5*i for i in X]
        a = 2/(sum([i**2 for i in X] )+1)
        X = [i*a for i in X]+[a-1]
        return [X,Z,Y]

    '''
    def grho_cal(self,theta):
        t = theta/(2*math.pi)
        return (t**4-2*(t**3)+t**2-1/30)*self.cc_B4

    def construct_data(self):
        theta = 2*math.pi*random.random()
        Y = self.grho_cal(theta)
        Z = Y + random.gauss(0,0.5)
        return [[np.cos(theta),np.sin(theta)],Z,Y]
    '''
    
    def updating(self,gamma0,t,theta,Sum_epoch,NN):
        self.gamma0 = gamma0
        self.t = t
        self.theta = theta
        start = time.perf_counter()
        self.Time = []
        self.Err = []
        for i in range(1,Sum_epoch+1):
            self.epoch = i
            Z = self.construct_data()
            self.iteration(Z[0],Z[-2])
            if i in NN:
                S1 = 0
                S2 = 0
                end = time.perf_counter()
                for j in range(1000):
                    Z = self.construct_data()
                    S1 +=(self.Computation_fn(Z[0])-Z[-1])**2
                    S2 +=(self.Computation_fn_average(Z[0])-Z[-1])**2
                runtime = end-start
                print('iteration:',i,S1/1000,S2/1000,self.L_n,runtime)
                self.Time.append(runtime)
                self.Err.append(S2/1000)
                #print(self.f_average_n)
                #print('K_n',self.K_n)
        return [self.Time,self.Err]
         

In [75]:
'non sphere data d=5'
d = 6
N = [int(10**(i/5))for i in range(1,36)]
s = 0.6
gamma0 =1.2
t = 0.05
theta = 1/(2*s+1)
Sum_epoch = int(10**7+1)
print(Sum_epoch)
t_kernel_SGD = T_kernel_SGD(d,s)

t_kernel_SGD.updating(gamma0,t,theta,Sum_epoch,N)

10000001
iteration: 1 25.63072119316037 77.75897172434014 0 0.0001759999431669712
iteration: 2 18.187024189830403 42.14443453320322 0 0.03563100006431341
iteration: 3 26.7032370081764 38.784920647669885 0 0.06413099984638393
iteration: 6 31.572297245951532 19.44623499318294 1 0.09152849996462464
iteration: 10 57.80522533054379 19.90479747471634 1 0.155379599891603
iteration: 15 18.85892398006487 20.716804096322104 1 0.20974109997041523
iteration: 25 90.10563962022101 16.88098556326661 1 0.27472230000421405
iteration: 39 39.29031363743655 16.201675447818563 1 0.3386558999773115
iteration: 63 9.685304669727032 11.052191851959883 1 0.4084874999243766
iteration: 100 9.653163434003455 8.767135046050052 2 0.48564519989304245
iteration: 158 5.114908018716202 5.922118305167949 2 0.7130652999039739
iteration: 251 3.308457377352336 3.947277583607314 2 0.9454548000358045
iteration: 398 9.199493297529427 3.22073981081672 2 1.186015299987048
iteration: 630 7.690790449943344 2.346994804736496 2 1.48

[[0.0001759999431669712,
  0.03563100006431341,
  0.06413099984638393,
  0.09152849996462464,
  0.155379599891603,
  0.20974109997041523,
  0.27472230000421405,
  0.3386558999773115,
  0.4084874999243766,
  0.48564519989304245,
  0.7130652999039739,
  0.9454548000358045,
  1.186015299987048,
  1.489430099958554,
  1.7785553999710828,
  2.157477800035849,
  2.854803799884394,
  3.784458499867469,
  5.202095799846575,
  7.215121899964288,
  10.637127900030464,
  17.10600350005552,
  26.71131799998693,
  42.77352269995026,
  71.30486000003293,
  118.77854169998318,
  187.65963779995218,
  297.55244189989753,
  499.94372250000015,
  832.4652378999162,
  1360.83064369997,
  2297.6631948000286,
  3869.7351939000655,
  6349.86361910007,
  10856.101079999935],
 [77.75897172434014,
  42.14443453320322,
  38.784920647669885,
  19.44623499318294,
  19.90479747471634,
  20.716804096322104,
  16.88098556326661,
  16.201675447818563,
  11.052191851959883,
  8.767135046050052,
  5.922118305167949,
  

In [76]:
'non sphere data d=20'
d = 21
N = [int(10**(i/5))for i in range(1,36)]
s = 0.6
gamma0 =1.2
t = 0.05
theta = 1/(2*s+1)
Sum_epoch = int(10**7+1)
print(Sum_epoch)
t_kernel_SGD = T_kernel_SGD(d,s)

t_kernel_SGD.updating(gamma0,t,theta,Sum_epoch,N)

10000001
iteration: 1 16.675618291522362 58.6282626104999 0 0.00010860012844204903
iteration: 2 39.76846869858983 25.749056094720974 0 0.03414519992657006
iteration: 3 23.601272707913832 24.681980220375628 0 0.06378310010768473
iteration: 6 81.55632648962899 18.274156024786983 1 0.09396610013209283
iteration: 10 162.23325217648005 22.890788270431166 1 0.22292920015752316
iteration: 15 26.02207006005434 15.36731831198804 1 0.3523977000731975
iteration: 25 39.47846501774096 14.42348870311819 1 0.48720300011336803
iteration: 39 22.505107478744733 14.024757907577394 1 0.6196044001262635
iteration: 63 18.792647986691552 11.88558476431507 1 0.767012600088492
iteration: 100 11.00691737893157 10.365676605193867 1 0.9028336000628769
iteration: 158 6.287135266389576 8.697870894470253 1 1.0485763000324368
iteration: 251 4.518679424371061 6.410095724234458 1 1.2012291001155972
iteration: 398 29.453621994594627 5.038592889286464 1 1.3810992001090199
iteration: 630 3.575555536188974 4.33352626763490

[[0.00010860012844204903,
  0.03414519992657006,
  0.06378310010768473,
  0.09396610013209283,
  0.22292920015752316,
  0.3523977000731975,
  0.48720300011336803,
  0.6196044001262635,
  0.767012600088492,
  0.9028336000628769,
  1.0485763000324368,
  1.2012291001155972,
  1.3810992001090199,
  1.5738546999637038,
  1.8116681000683457,
  2.8184804001357406,
  4.091773699969053,
  5.8554198001511395,
  8.249978200066835,
  11.77811810001731,
  17.025538800051436,
  25.06668039993383,
  37.517542799934745,
  56.90860480000265,
  87.8321913999971,
  135.6984695000574,
  244.0344963001553,
  444.8605655001011,
  763.6623118000571,
  1267.2456561001018,
  2065.723809200106,
  3329.3248072999995,
  5329.465881299926,
  8498.27776680002,
  13565.81389739993],
 [58.6282626104999,
  25.749056094720974,
  24.681980220375628,
  18.274156024786983,
  22.890788270431166,
  15.36731831198804,
  14.42348870311819,
  14.024757907577394,
  11.88558476431507,
  10.365676605193867,
  8.697870894470253,
 

In [45]:
def construct_data(d):
    X = [random.uniform(0,1) for i in range(d-1)]
    Y = 10*np.sin(X[0]*X[1])+20*(X[2]-0.5)**2+10*X[3]+5*X[4]+X[1]*X[2]*X[3]+np.cos(X[2]*X[3])
    Z = Y+random.gauss(0,0.5)
    X = [ 0.7*i for i in X]
    a = 2/(sum([i**2 for i in X] )+1)
    X = [i*a for i in X]+[a-1]
    return [X,Z,Y]
for i in range(1):
    print(construct_data(6))

[[0.5371942647518464, 0.4787840541495438, 0.3713114132990481, 0.14083537413845698, 0.5274587608858101, -0.2151014614543566], 16.02272401064172, 16.64366081225225]
